In [40]:
@everywhere push!(LOAD_PATH, "C:\\Users\\LinFan Xiao\\Academics\\College\\Capstone\\work_in_julia")
include("./construct_adjoint.jl")
using construct_adjoint

In [13]:
t = symbols("t")

t

In [14]:
symL = SymLinearDifferentialOperator([t+1 t+1 t+1], (0,1), t)

SymLinearDifferentialOperator(SymPy.Sym[t + 1 t + 1 t + 1], (0, 1), t)

In [15]:
L = LinearDifferentialOperator([t->t+1, t->t+1, t->t+1], (0,1), symL)

LinearDifferentialOperator(Function[#249, #250, #251], (0, 1), SymLinearDifferentialOperator(SymPy.Sym[t + 1 t + 1 t + 1], (0, 1), t))

In [18]:
u, v = SymFunction("u")(t), SymFunction("v")(t)

(u(t), v(t))

In [41]:
symUVForm = get_symUvForm(L, u, v; substitute = true)
pprint(symUVForm)

SymPy.Sym

       ____          d /____\     ____ d               d /____\   ____ d      
t*u(t)*v(t) - t*u(t)*--\v(t)/ + t*v(t)*--(u(t)) - u(t)*--\v(t)/ + v(t)*--(u(t)
                     dt                dt              dt              dt     

 
)
 


In [22]:
symUVForm = get_symUvForm(L, u, v; substitute = false)
pprint(symUVForm)

             d /____\         ____ d                     ____        ____ d   
- p0(t)*u(t)*--\v(t)/ + p0(t)*v(t)*--(u(t)) + p1(t)*u(t)*v(t) - u(t)*v(t)*--(p
             dt                    dt                                     dt  

     
0(t))
     


In [62]:
symDerivMatrix = get_symPDerivMatrix(L; substitute = true)

2×2 Array{SymPy.Sym,2}:
 t + 1  1
 t + 1  1

In [63]:
symDerivMatrix = get_symPDerivMatrix(L; substitute = false)

2×2 Array{SymPy.Sym,2}:
 p0(t)  Derivative(p0(t), t)
 p1(t)  Derivative(p1(t), t)

In [23]:
pDerivMatrix = [t->t+1 t->t; t->t+1 t->t]

2×2 Array{Function,2}:
 #255  #256
 #257  #258

In [66]:
B = get_B(L; pDerivMatrix = pDerivMatrix)

2×2 Array{Union{Function, Number},2}:
 h   h
 h  0 

In [36]:
symB = get_B(L; symbolic = true)

2×2 Array{Any,2}:
      t  t + 1
 -t - 1      0

In [47]:
symB = get_B(L; symbolic = true, substitute = false)

2×2 Array{Union{Function, Number},2}:
 p1(t) - Derivative(p0(t), t)  p0(t)
                       -p0(t)      0

In [48]:
BHat = get_BHat(L, B)

4×4 Array{Number,2}:
 -1  -1   0  0
  1   0   0  0
  0   0   1  2
  0   0  -2  0

In [53]:
symXi = get_symXi(L; substitute = true, xDef = t^2+2)

2×1 Array{SymPy.Sym,2}:
 t^2 + 2
     2*t

In [50]:
evaluate_xi(L, 1, symXi)

2×1 Array{Number,2}:
 3
 2

In [54]:
xi = [t->t^2+2; t->2t]

2-element Array{Function,1}:
 #325
 #326

In [55]:
evaluate_xi(L, 1, xi)

2×1 Array{Number,2}:
 3
 2

In [56]:
U = VectorBoundaryForm([1 2; 3 4], [4 3; 2 1])
get_boundaryCondition(L, U, symXi)

2×1 Array{SymPy.Sym,2}:
 20
 14

In [57]:
get_boundaryCondition(L, U, xi)

2×1 Array{Int64,2}:
 20
 14

In [58]:
# Test the algorithm to generate valid adjoint U+
function test_generate_adjoint(n, k)
    results = [true]
    t = symbols("t")
    (a,b) = (0,1)

    println("Testing the algorithm to generate valid adjoint U+: Constant p_k")
    pFunctions = rand(Uniform(1.0,10.0), 1, (n+1))
    symPFunctions = pFunctions
    symL = SymLinearDifferentialOperator(symPFunctions, (a,b), t)
    L = LinearDifferentialOperator(pFunctions, (a,b), symL)
    MCand = rand(Uniform(1.0,10.0), n, n)
    NCand = rand(Uniform(1.0,10.0), n, n)
    U = VectorBoundaryForm(MCand, NCand)
    pDerivMatrix = eye(n)
    println("Testing: order of L = $n")
    for counter = 1:k
        for i = 1:n
            for j = 1:n
                if j == 1
                    pDerivMatrix[i,j] = pFunctions[i]
                else
                    pDerivMatrix[i,j] = 0
                end
            end
        end
        passed = false
        try
            adjoint = construct_validAdjoint(L, U, pDerivMatrix)
            passed = true
            append!(results, passed)
        catch err
            println("Failed with $err")
        end
        if passed
            println("Test $counter: Passed!")
        end
    end

    println("Testing the algorithm to generate valid adjoint U+: Variable p_k")
    # Generate variable p_k
    pFunctions = Array{Function}(1,n+1)
    symPFunctions = Array{Number}(1,n+1)
    pDerivMatrix = Array{Union{Function, Number}}(n,n)
    for i = 1:(n+1)
        # Each p_k is a polynomial function with random degree between 0 to 4 and random coefficients between 0 and 10
        pFunctionCoeffs = rand(Uniform(1.0,10.0), 1, (rand(1:5)))
        if i < n+1
            pDerivMatrix[i:i, 1:n] = [get_polynomialDeriv(pFunctionCoeffs, k) for k = 0:(n-1)]
        end
        pFunction = get_polynomial(pFunctionCoeffs)
        pFunctions[i] = pFunction
        symPFunction = sum([pFunctionCoeffs[i+1]*t^(length(pFunctionCoeffs)-1-i) for i in 0:(length(pFunctionCoeffs)-1)])
        symPFunctions[i] = symPFunction
    end
    symL = SymLinearDifferentialOperator(symPFunctions, (a,b), t)
    L = LinearDifferentialOperator(pFunctions, (a,b), symL)
    MCand = rand(Uniform(1.0,10.0), n, n)
    NCand = rand(Uniform(1.0,10.0), n, n)
    U = VectorBoundaryForm(MCand, NCand)
    println("Testing: order of L = $n")
    for counter = 1:k
        for i = 1:n
            for j = 1:n
                if j == 1
                    pDerivMatrix[i,j] = pFunctions[i]
                else
                    pDerivMatrix[i,j] = 0
                end
            end
        end
        passed = false
        try
            adjoint = construct_validAdjoint(L, U, pDerivMatrix)
            passed = true
            append!(results, passed)
        catch err
            println("Failed with $err")
        end
        if passed
            println("Test $counter: Passed!")
        end
    end

    return all(results)
end
for n = 1:10
    println(test_generate_adjoint(n, 10))
end

Testing the algorithm to generate valid adjoint U+: Constant p_k
Testing: order of L = 1
Test 1: Passed!
Test 2: Passed!
Test 3: Passed!
Test 4: Passed!
Test 5: Passed!
Test 6: Passed!
Test 7: Passed!
Test 8: Passed!
Test 9: Passed!
Test 10: Passed!
Testing the algorithm to generate valid adjoint U+: Variable p_k
Testing: order of L = 1
Test 1: Passed!
Test 2: Passed!
Test 3: Passed!
Test 4: Passed!
Test 5: Passed!
Test 6: Passed!
Test 7: Passed!
Test 8: Passed!
Test 9: Passed!
Test 10: Passed!
true
Testing the algorithm to generate valid adjoint U+: Constant p_k
Testing: order of L = 2
Test 1: Passed!
Test 2: Passed!
Test 3: Passed!
Test 4: Passed!
Test 5: Passed!
Test 6: Passed!
Test 7: Passed!
Test 8: Passed!
Test 9: Passed!
Test 10: Passed!
Testing the algorithm to generate valid adjoint U+: Variable p_k
Testing: order of L = 2
Test 1: Passed!
Test 2: Passed!
Test 3: Passed!
Test 4: Passed!
Test 5: Passed!
Test 6: Passed!
Test 7: Passed!
Test 8: Passed!
Test 9: Passed!
Test 10: Pas

In [59]:
# Test the SymLinearDifferentialOperator definition
function test_symLinearDifferentialOperatorDef()
    results = [true]
    println("Testing definition of SymLinearDifferentialOperator: symP_k are SymPy.Sym")
    t = symbols("t")
    passed = false
    try
        SymLinearDifferentialOperator([t+1 t+1 t+1], (0,1), t)
        passed = true
    catch err
        println("Failed with $err")
    end
    if passed
        println("Passed!")
    end
    append!(results, passed)

    println("Testing definition of SymLinearDifferentialOperator: symP_k are Number")
    passed = false
    try
        SymLinearDifferentialOperator([1 1 1], (0,1), t)
        passed = true
    catch err
        println("Failed with $err")
    end
    if passed
        println("Passed!")
    end
    append!(results, passed)

    println("Testing definition of SymLinearDifferentialOperator: symP_k are SymPy.Sym and Number")
    passed = false
    try
        SymLinearDifferentialOperator([1 1 t+1], (0,1), t)
        passed = true
    catch err
        println("Failed with $err")
    end
    if passed
        println("Passed!")
    end
    append!(results, passed)

    println("Testing StructDefinitionError: symP_k should be SymPy.Sym or Number")
    passed = false
    try
        SymLinearDifferentialOperator(['s' 1 t+1], (0,1), t)
    catch err
        if isa(err,StructDefinitionError) && err.msg == "symP_k should be SymPy.Sym or Number"
            passed = true
            println("Passed!")
        else
            println("Failed with $err")
        end
    end
    if !passed
        println("Failed!")
    end
    append!(results, passed)

    println("Testing StructDefinitionError: Only one free symbol is allowed in symP_k")
    a = symbols("a")
    passed = false
    try
        SymLinearDifferentialOperator([t+1 t+1 a*t+1], (0,1), t)
    catch err
        if isa(err,StructDefinitionError) && err.msg == "Only one free symbol is allowed in symP_k"
            passed = true
            println("Passed!")
        else
            println("Failed with $err")
        end
    end
    if !passed
        println("Failed!")
    end
    append!(results, passed)

    return all(results)
end
test_symLinearDifferentialOperatorDef()

Testing definition of SymLinearDifferentialOperator: symP_k are SymPy.Sym
Passed!
Testing definition of SymLinearDifferentialOperator: symP_k are Number
Passed!
Testing definition of SymLinearDifferentialOperator: symP_k are SymPy.Sym and Number
Passed!
Testing StructDefinitionError: symP_k should be SymPy.Sym or Number
Passed!
Testing StructDefinitionError: Only one free symbol is allowed in symP_k
Passed!


true

In [60]:
# Test the LinearDifferentialOperator definition
function test_linearDifferentialOperatorDef()
    results = [true]
    # Variable p_k
    println("Testing definition of LinearDifferentialOperator: p_k are variable Function")
    t = symbols("t")
    symL = SymLinearDifferentialOperator([t+1 t+1 t+1], (1,2), t)
    passed = false
    try
        L = LinearDifferentialOperator([t->t+1 t->t+1 t->t+1], (1,2), symL)
        passed = true
    catch err
        println("Failed with $err")
    end
    if passed
        println("Passed!")
    end
    append!(results, passed)

    # Constant coefficients
    println("Testing definition of LinearDifferentialOperator: p_k are Number")
    symL = SymLinearDifferentialOperator([1 1 1], (0,1), t)
    passed = false
    try
        LinearDifferentialOperator([1 1 1], (0,1), symL)
        passed = true
    catch err
        println("Failed with $err")
    end
    if passed
        println("Passed!")
    end
    append!(results, passed)

    println("Testing definition of LinearDifferentialOperator: p_k are constant Function")
    symL = SymLinearDifferentialOperator([1 1 1], (0,1), t)
    passed = false
    try
        LinearDifferentialOperator([t->1 t->1 t->1], (0,1), symL)
        passed = true
    catch err
        println("Failed with $err")
    end
    if passed
        println("Passed!")
    end
    append!(results, passed)

    # Mixed coefficients
    println("Testing definition of LinearDifferentialOperator: p_k are mixed")
    symL = SymLinearDifferentialOperator([1 1 t+1], (0,1), t)
    passed = false
    try
        LinearDifferentialOperator([1 t->1 t->t+1], (0,1), symL)
        passed = true
    catch err
        println("Failed with $err")
    end
    if passed
        println("Passed!")
    end
    append!(results, passed)

    println("Testing StructDefinitionError: p_k should be Function or Number")
    passed = false
    try
        LinearDifferentialOperator(['s' 1 1], (0,1), symL)
    catch err
        if err.msg == "p_k should be Function or Number" && (isa(err,StructDefinitionError))
            passed = true
            println("Passed!")
        else
            println("Failed with $err")
        end
    end
    if !passed
        println("Failed!")
    end
    append!(results, passed)

    println("Testing StructDefinitionError: Number of p_k and symP_k do not match")
    symL = SymLinearDifferentialOperator([1 1 t+1], (0,1), t)
    passed = false
    try
        LinearDifferentialOperator([1 t->1], (0,1), symL)
    catch err
        if err.msg == "Number of p_k and symP_k do not match" && (isa(err, StructDefinitionError))
            passed = true
            println("Passed!")
        else
            println("Failed with $err")
        end
    end
    if !passed
        println("Failed!")
    end
    append!(results, passed)

    println("Testing StructDefinitionError: p0 vanishes on [a,b]")
    function p2(t) return t end
    symL = SymLinearDifferentialOperator([t 1 2], (0,1), t)
    passed = false
    try
        LinearDifferentialOperator([t->t 1 2], (0,1), symL)
    catch err
        if err.msg == "p0 vanishes on [a,b]" && (isa(err, StructDefinitionError))
            passed = true
            println("Passed!")
        else
            println("Failed with $err")
        end
    end
    if !passed
        println("Failed!")
    end
    append!(results, passed)
    
    # # This is now a warning
    # println("Testing StructDefinitionError: symP_k does not agree with p_k on [a,b]")
    # symL = SymLinearDifferentialOperator([t+1 t+1 t+2], (0,1), t)
    # passed = false
    # try
    #     LinearDifferentialOperator([t->t+1 t->t+1 t->t+1], (0,1), symL)
    # catch err
    #     if err.msg == "symP_k does not agree with p_k on [a,b]" && (isa(err,StructDefinitionError))
    #         passed = true
    #         println("Passed!")
    #     else
    #         println("Failed with $err")
    #     end
    # end
    # if !passed
    #     println("Failed!")
    # end
    # append!(results, passed)

    return all(results)
end
test_linearDifferentialOperatorDef()

Testing definition of LinearDifferentialOperator: p_k are variable Function
Passed!
Testing definition of LinearDifferentialOperator: p_k are Number
Passed!
Testing definition of LinearDifferentialOperator: p_k are constant Function
Failed with ErrorException("type Int64 has no field re")
Testing definition of LinearDifferentialOperator: p_k are mixed
Passed!
Testing StructDefinitionError: p_k should be Function or Number
Passed!
Testing StructDefinitionError: Number of p_k and symP_k do not match
Passed!
Testing StructDefinitionError: p0 vanishes on [a,b]
Passed!


false

In [61]:
# Test the VectorBoundaryForm definition
function test_vectorBoundaryFormDef()
    results = [true]
    println("Testing the definition of VectorBoundaryForm")
    M = eye(3)
    N = M
    passed = false
    try
        VectorBoundaryForm(M, N)
        passed = true
    catch err
        println("Failed with $err")
    end
    if passed
        println("Passed!")
    end
    append!(results, passed)

    println("Testing StructDefinitionError: Entries of M, N should be Number")
    M = ['a' 2; 3 4]
    N = M
    passed = false
    try
        VectorBoundaryForm(M, N)
    catch err
        if err.msg == "Entries of M, N should be Number" && isa(err, StructDefinitionError)
            passed = true
            println("Passed!")
        else
            println("Failed with $err")
        end
    end
    if !passed
        println("Failed!")
    end
    append!(results, passed)

    println("Testing StructDefinitionError: M, N dimensions do not match")
    M = eye(2)
    N = eye(3)
    passed = false
    try
        VectorBoundaryForm(M, N)
    catch err
        if err.msg == "M, N dimensions do not match" && isa(err,StructDefinitionError)
            passed = true
            println("Passed!")
        else
            println("Failed with $err")
        end
    end
    if !passed
        println("Failed!")
    end
    append!(results, passed)

    println("Testing StructDefinitionError: M, N should be square matrices")
    M = [1 2]
    N = M
    passed = false
    try
        VectorBoundaryForm(M, N)
    catch err
        if err.msg == "M, N should be square matrices" && isa(err,StructDefinitionError)
            passed = true
            println("Passed!")
        else
            println("Failed with $err")
        end
    end
    if !passed
        println("Failed!")
    end
    append!(results, passed)

    println("Testing StructDefinitionError: Boundary operators not linearly independent")
    M = [1 2; 2 4]
    N = [3 4; 6 8]
    passed = false
    try
        VectorBoundaryForm(M, N)
    catch err
        if err.msg == "Boundary operators not linearly independent" && isa(err,StructDefinitionError)
            passed = true
            println("Passed!")
        end
    end
    if !passed
        println("Failed!")
    end
    append!(results, passed)

    return all(results)
end
test_vectorBoundaryFormDef()

Testing the definition of VectorBoundaryForm
Passed!
Testing StructDefinitionError: Entries of M, N should be Number
Passed!
Testing StructDefinitionError: M, N dimensions do not match
Passed!
Testing StructDefinitionError: M, N should be square matrices
Passed!
Testing StructDefinitionError: Boundary operators not linearly independent
Passed!


true